# Daniilidis calibration

In [2]:
from __future__ import print_function

In [3]:
import sys
sys.path.append('../build/')

In [4]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [5]:
import versor as vsr

## Generate dataset

In [6]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(0.1, 0.2, 0.3).trs() * vsr.Biv(0.0,1.0,0.0).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

In [24]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
#             LAs.append(((As[j].rev() * As[i]).log() ))
#             LBs.append(((Bs[j] * Bs[i].rev()).log() ))
            LAs.append(vsr.Dll(*np.array(As[j].rev() * As[i])[1:7]))
            LBs.append(vsr.Dll(*np.array(Bs[j] * Bs[i].rev())[1:7]))
    return LAs, LBs

In [25]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)
print(len(lines_a))

45


In [26]:
np.ones(9)[1:7]

array([ 1.,  1.,  1.,  1.,  1.,  1.])

In [27]:
from scipy import linalg

In [28]:
def daniilidis_calibration(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
#         LA = LA.unit()
#         LB = LB.unit()
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]

        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    
#     [U, s, Vt] = linalg.svd(Ds, full_matrices=True)
    [U, s, Vt] = np.linalg.svd(Ds)

    v7 = Vt.T[:,-2]
    v8 = Vt.T[:,-1]
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * v7 + lambda2 * v8
    m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
    m_arr[4:8] = np.roll(m_arr[4:8], -1)
    
    return vsr.Mot(*m_arr), Ds

In [29]:
estimated_motor, D = daniilidis_calibration(lines_b, lines_a)

## Check solution

In [30]:
estimated_motor * estimated_motor.rev()

Mot: [ 1 0 -8.5e-33 0 -4.2e-18 0 -1.5e-18 0.18 ]

In [31]:
estimated_motor.matrix()

array([[-0.4161, -0.    , -0.9093,  0.1   ],
       [-0.    ,  1.    , -0.    , -0.0832],
       [ 0.9093,  0.    , -0.4161,  0.3   ],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [37]:
M_eye_in_hand.matrix()

array([[-0.4161,  0.    , -0.9093,  0.1   ],
       [ 0.    ,  1.    ,  0.    ,  0.2   ],
       [ 0.9093,  0.    , -0.4161,  0.3   ],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [26]:
function [Hcam2marker_, err] = hand_eye_dual_quaternion(Hmarker2world, Hgrid2cam)
%Our quaternions are like this (q1 q2 q3,s )
    %Get n
    n = size(Hmarker2world,3);
    %Make movements (a,B) which are interposition transformations
    %(marker2wordl and cam2grid)
    %transform A,B into dual quaternions        
    for i=1:n-1
        A = inv(Hmarker2world(:,:,i+1))*Hmarker2world(:,:,i);
        B = Hgrid2cam(:,:,i+1)*inv(Hgrid2cam(:,:,i));         
        [q,qprime] = getDualQuaternion(A(1:3,1:3),A(1:3,4));               
        Qa(i).q = q;
        Qa(i).qprime = qprime;

        [q,qprime] = getDualQuaternion(B(1:3,1:3),B(1:3,4));                
        Qb(i).q = q;
        Qb(i).qprime = qprime;
    end
    
    %The dual quaternion is (Q.q + epsilon*Q.prime)
    %a = Qa.q, a' = Qa.prime  idem for b
    S = [];
    for i=1:n-1
        S(:,:,i) = [Qa(i).q(1:3)-Qb(i).q(1:3)   crossprod(Qa(i).q(1:3)+Qb(i).q(1:3)) zeros(3,1) zeros(3,3);...
                    Qa(i).qprime(1:3)-Qb(i).qprime(1:3)   crossprod(Qa(i).qprime(1:3)+Qb(i).qprime(1:3)) Qa(i).q(1:3)-Qb(i).q(1:3)   crossprod(Qa(i).q(1:3)+Qb(i).q(1:3))];                                
    end  
    
    %Construct T
    T = [];    
    for i=1:n-1
        T = [T  S(:,:,i)'];      
    end
    
    T = T';
    %SVD 
    [U,S,V] = svd(T);
    
    %Solution, right null vectors of T
    v7 = V(:,7);
    v8 = V(:,8);
    
    u1 = v7(1:4);
    v1 = v7(5:8);
    
    u2 = v8(1:4);
    v2 = v8(5:8);
    %Now lambda1*v7+lambda2*v8 = [q;qprime]
    %
    %or other:
    %
    %lambda1^2*u1'*u1+2*lambda1*lambda2*u1'*u2+lambda2^2*u2'*u2 = 1   
    %and
    %lambda1^2*u1'*v1 + lambda1*lambda2*(u1'*v2+u2'*v1)+lambda2*u2'*v1 = 0
    %Setting lambda1/lambda2 = s
    %lambda1^2/lambda2^2*u1'*v1 + lambda1*lambda2/lambda2^2*(u1'*v2+u2'*v1)+lambda2^2/lambda2^2*u2'*v1 = 0
    %s^2*u1'*v1 + s*(u1'*v2+u2'*v1)+u2'*v1 = 0
    %s^2*u1'*v1 + s*(u1'*v2+u2'*v1)+u2'*v1 = 0
    a = u1'*v1;
    b = (u1'*v2+u2'*v1);
%     c = u2'*v1;
    c = u2'*v2 ;
    s = roots([a b c]);
    
    %insert into equation
    val1 = s(1)^2*u1'*u1+2*s(1)*u1'*u2+u2'*u2;
    val2 = s(2)^2*u1'*u1+2*s(2)*u1'*u2+u2'*u2;
    %Take bigger value
    if(val1>val2)
        s = s(1);
        val = val1;
    else
        s = s(2);
        val = val2;
    end
    %Get lambdas
    lambda2 = sqrt(1/val);
    lambda1 = s*lambda2;
    
    %This algorithm gives quaternion with the form of (s, q1 q2
    %q3)->contrary to the notation we used above (q1 q2 q3,s )
    %Therefore we must rearrange the elements!        
    qfinal = lambda1*v7+lambda2*v8;    
    q = [qfinal(2:4);qfinal(1)];
    qprime = [qfinal(6:8);qfinal(5)];
    
    %Extract transformation
    R = q2dcm(q);    
    t = 2*qmult(qprime,qconj(q));
    t = t(1:3);
    
    %Assign output arguments
    Hcam2marker_ = [R -R*t;[0 0 0 1]]^-1;    
    err=[];
    


    



IndentationError: unexpected indent (<ipython-input-26-ca12279846f6>, line 3)

In [ ]:
#Creates a dual quaternion from a rotation matrix and a translation vector    
def getDualQuaternion(R,t)    
    %Conversion from R,t to the screw representation [d,theta,l,m]
    
    r = rodrigues(R);
    theta = norm(r);
    l = r/norm(theta);
    %Pitch d
    d = l'*t;    
    %Make point c
    c = .5*(t-d*l)+cot(theta/2)*cross(l,t);
    %moment vector
    m = cross(c,l);
    %Rotation quaternion
    %(q1 q2 q3,s )
    q = [sin(theta/2)*l; cos(theta/2)];
    %Get dual
    qprime = [.5*(q(4)*t+cross(t,q(1:3)));-.5*q(1:3)'*t];